# Getting Started with Monarch
This notebook demonstrates a simple interactive PV loop visualization using the Monarch cardiac model.

In [8]:
from monarch import Hatch
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import pathlib
import monarch.metamorphoses as meta

In [9]:
input_dir = pathlib.Path.cwd() / 'input_files'
input_human = input_dir / 'human'

In [10]:
def interactive_pv_loop(hr=60, sbv=1500, contractility=0.15):
    beat_mod = Hatch(input_human)
    beat_mod.change_pars({
        "HR": hr,
        "sbv": sbv,
    })
    beat_mod.heart.sf_act[0] = contractility  # LFW contractility
    beat_mod.just_beat_it()

    #plt.figure(figsize=(10, 6))
    # meta.pv_loop(beat_mod, compartments=("LV", "RV"))
    # plt.title(f"HR: {hr} bpm | SBV: {sbv} mL | Contractility: {contractility:.2f}")
    # plt.show()
    #meta.pv_loop(beat_mod, compartments=("LV", "RV"))
    #plt.title(f"HR: {hr} bpm | SBV: {sbv} mL | Contractility: {contractility:.2f}")
    #plt.show()


In [4]:

hr_slider = widgets.IntSlider(value=60, min=40, max=120, step=5, description="HR:")
sbv_slider = widgets.IntSlider(value=1500, min=1000, max=2000, step=50, description="SBV:")
contractility_slider = widgets.FloatSlider(value=0.15, min=0.05, max=0.3, step=0.01, description="Contractility")
#
ui = widgets.VBox([hr_slider, sbv_slider, contractility_slider])
out = widgets.interactive_output(interactive_pv_loop, {
    "hr": hr_slider,
    "sbv": sbv_slider,
    "contractility": contractility_slider
})
#display(ui, out)


In [5]:
### Extended: Interactive Geometry + PV Loop with Stiffness and Wall Volume
import ipywidgets as widgets
import numpy as np

def interactive_pv_loop_extended(hr=60, sbv=1500, lv_contractility=0.15, septal_contractility=0.15, stiffness=0.017, wall_volume=40000):
    beat_mod = Hatch(input_human)
    beat_mod.change_pars({
        "HR": hr,
        "sbv": sbv
    })

    # Set contractility
    beat_mod.heart.sf_act[0] = lv_contractility   # LFW
    beat_mod.heart.sf_act[2] = septal_contractility  # Septal Wall

    # Set wall volume
    beat_mod.heart.vw[0] = wall_volume  # LFW volume

    # Set stiffness parameters
    beat_mod.heart.c_3 = np.full_like(beat_mod.heart.c_3, stiffness)
    beat_mod.heart.c_4 = np.full_like(beat_mod.heart.c_4, stiffness * 600)

    # Run simulation
    beat_mod.just_beat_it()

    fig = plt.figure(figsize=(14, 6))
    # PV Loop
    meta.pv_loop(beat_mod, compartments=("LV", "RV"))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.set_title(f"HR: {hr} bpm | SBV: {sbv} mL")
    meta.pv_loop(beat_mod, compartments=("LV", "RV"))

    # Geometry
    meta.cardiac_geometry(beat_mod, real_wall_thickness=True)
    ax1.set_title(f"Geometry (Vol: {wall_volume} mm³, Stiffness: {stiffness:.3f})")

    plt.tight_layout()
    #plt.show()



# Create extended sliders
stiffness_slider = widgets.FloatSlider(value=0.013, min=0.001, max=0.02, step=0.001, description="Stiffness:")
wall_volume_slider = widgets.IntSlider(value=40000, min=10000, max=80000, step=1000, description="Wall Volume:")
lv_contractility_slider = widgets.FloatSlider(
    value=0.15, min=0.05, max=0.3, step=0.01, description="LV Contractility"
)
septal_contractility_slider = widgets.FloatSlider(
    value=0.15, min=0.05, max=0.3, step=0.01, description="Septal Contractility"
)

ui_ext = widgets.VBox([
    hr_slider, sbv_slider, lv_contractility_slider, septal_contractility_slider,
    stiffness_slider, wall_volume_slider
])
out_ext = widgets.interactive_output(interactive_pv_loop_extended, {
    "hr": hr_slider,
    "sbv": sbv_slider,
    "lv_contractility": lv_contractility_slider,
    "septal_contractility": septal_contractility_slider,
    "stiffness": stiffness_slider,
    "wall_volume": wall_volume_slider
})
display(ui_ext, out_ext)


lv_contractility_slider = widgets.FloatSlider(value=0.15, min=0.05, max=0.3, step=0.01, description="LV Contractility")
septal_contractility_slider = widgets.FloatSlider(value=0.15, min=0.05, max=0.3, step=0.01, description="Septal Contractility")


Output()